<a href="https://colab.research.google.com/github/changosue/Analisis-estadisticos-en-R/blob/main/Distribucion_especies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualización de la Distribución de Especies utilizando GBIF y Google Colab

**Objetivo:**

Este cuaderno de Colab te permitirá visualizar la distribución geográfica de una especie de tu elección utilizando datos de GBIF (una base de datos global de biodiversidad). Además, podrás descargar los datos visualizados en diferentes formatos.

## **Paso 1: Preparación**

**Instalación de bibliotecas necesarias:**

Antes de comenzar, necesitamos instalar algunas herramientas que nos ayudarán a procesar y visualizar los datos. Simplemente ejecuta la siguiente celda:

In [ ]:
!pip install pandas plotly geopandas
!pip install -U kaleido



**Importar bibliotecas:**

Una vez que las herramientas estén instaladas, debemos "importarlas" para utilizarlas. Piensa en ello como encender una máquina antes de usarla.


In [ ]:
import requests
import pandas as pd
import plotly.graph_objects as go
import geopandas as gpd
from shapely.geometry import Point
from google.colab import files
import zipfile
import os



##**Paso 2: Obtener y procesar datos**

**Función para obtener datos de GBIF:**

Esta función actúa como un mensajero, yendo al sitio web de GBIF, buscando la especie que queremos y trayendo los datos de vuelta para nosotros.

In [ ]:
def obtener_datos_gbif(nombre_especie, limite=3000):
    url_base = "https://api.gbif.org/v1/occurrence/search"
    parametros = {
        "q": nombre_especie,
        "limit": limite
    }

    respuesta = requests.get(url_base, params=parametros)

    if respuesta.status_code != 200:
        print("Error obteniendo datos.")
        return None

    data = respuesta.json()
    resultados = []

    for registro in data['results']:
        if 'decimalLatitude' in registro and 'decimalLongitude' in registro:
            resultados.append({
                'latitud': registro['decimalLatitude'],
                'longitud': registro['decimalLongitude'],
                'pais': registro['country'] if 'country' in registro else None
            })

    df = pd.DataFrame(resultados)
    return df

**Función para formatear la lista de países:**

Si la especie que buscas está en muchos países, esta función nos ayudará a mostrar esa lista de una manera legible.

In [ ]:
def dividir_paises(lista_paises, max_chars_por_linea=50):
    palabras = lista_paises.split(", ")
    lineas = []
    linea_actual = []

    longitud_linea_actual = 0

    for palabra in palabras:
        if longitud_linea_actual + len(palabra) > max_chars_por_linea:
            lineas.append(", ".join(linea_actual))
            linea_actual = []
            longitud_linea_actual = 0

        linea_actual.append(palabra)
        longitud_linea_actual += len(palabra) + 2  # 2 for ", "

    # Añadir la última línea si tiene contenido
    if linea_actual:
        lineas.append(", ".join(linea_actual))

    return "<br>".join(lineas)

##**Paso 3: Visualizar datos**

**Función para crear mapa de calor:**

Esta función toma los datos que obtuvimos y crea una bonita visualización geográfica llamada "mapa de calor". Las áreas con más registros de la especie aparecerán más "calientes" o más brillantes.

In [ ]:
def mapa_calor_especie(nombre_especie):
    df = obtener_datos_gbif(nombre_especie)

    if df is None or df.empty:
        print(f"No se encontraron datos para {nombre_especie}.")
        return

    # Creando una columna de recuento
    df['recuento'] = 1

    # Definir coordenadas para centrar en México
    center_lat = 23.6345
    center_lon = -102.5528
    zoom_level = 4
    mapbox_style = "stamen-terrain"

    fig = go.Figure()

    # Tracer el mapa de densidad
    fig.add_trace(go.Densitymapbox(lat=df['latitud'],
                                   lon=df['longitud'],
                                   z=df['recuento'],
                                   radius=10,
                                   colorscale="Viridis",
                                   showscale=True,
                                   colorbar_title="Número de ocurrencias"))

    # Configuración general
    fig.update_layout(mapbox_style=mapbox_style,
                      mapbox_center_lon=center_lon,
                      mapbox_center_lat=center_lat,
                      mapbox_zoom=zoom_level)

    # Estadísticas y título
    titulo = f"Distribución de <i>{nombre_especie}</i>"

    en_mexico = "Sí" if "Mexico" in df['pais'].unique() else "No"
    distribucion = df['pais'].value_counts().to_dict()
    lista_paises = ", ".join(distribucion.keys())
    lista_paises_multilinea = dividir_paises(lista_paises)

    annotations = [
        dict(x=0.5, y=-0.2, xref='paper', yref='paper', showarrow=False, align='center',
             text=f"Resumen para <i>{nombre_especie}</i>:<br>"
                  f"Presente en México: {en_mexico}<br>"
                  f"Distribución por países: {lista_paises_multilinea}")
    ]

    fig.update_layout(annotations=annotations, title=titulo)

    fig.show()
    return fig, df


##**Paso 4: Descargar datos**

Quizás desees guardar los datos visualizados para usarlos más tarde o compartirlos. La siguiente funciones te permiten hacer exactamente eso.

In [ ]:
def descargar_jpeg(nombre_especie, fig):
    import plotly.io as pio
    pio.write_image(fig, 'mapa.jpeg', width=1920, height=1080)
    files.download('mapa.jpeg')


**Función para descargar como archivo CSV** (una especie de hoja de cálculo):

In [ ]:
def descargar_csv(df):
    df.to_csv("datos.csv", index=False)
    files.download('datos.csv')

**Función para descargar como archivo Shapefile** (formato utilizado en geografía y sistemas de información geográfica):

In [ ]:
def descargar_shp(df):
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud))
    gdf.crs = "EPSG:4326"
    gdf.to_file("datos.shp")

    # Comprimir todos los archivos relacionados en un archivo ZIP
    archivos_shapefile = ['datos.shp', 'datos.shx', 'datos.dbf', 'datos.prj', 'datos.cpg']
    with zipfile.ZipFile('datos.zip', 'w') as zipf:
        for archivo in archivos_shapefile:
            if os.path.exists(archivo):
                zipf.write(archivo)
            else:
                print(f"Advertencia: {archivo} no encontrado.")

    # Descargar el archivo ZIP
    files.download('datos.zip')

## **Paso 5: ¡Ponlo todo en marcha!**

Esta es la parte emocionante. Ahora vas a interactuar con el código:



*   Se te pedirá que ingreses el nombre de una especie.
*   Verás un mapa de calor de la distribución de esa especie.
*  Tendrás la opción de descargar los datos visualizados.

Simplemente ejecuta la siguiente celda y sigue las instrucciones:

In [ ]:
nombre_especie_input = input("Introduce el nombre de la especie para visualizar su mapa de calor: ")
fig, df = mapa_calor_especie(nombre_especie_input)


**Descarga de Datos:**

Después de visualizar el mapa de distribución de la especie, es posible que desees conservar estos datos para análisis posteriores o para compartirlos con colegas o publicarlos.

Por eso, te ofrecemos la opción de descargar los datos en diversos formatos:

* JPEG: Es una imagen del mapa visualizado. Ideal para presentaciones o informes.

* CSV: Formato "Comma Separated Values". Es una tabla que puedes abrir con programas de hoja de cálculo como Excel. Es útil si deseas analizar los datos con otras herramientas o software.

* SHP: Es el formato estándar de archivo para software de Sistemas de Información Geográfica (SIG). Te permite integrar estos datos en mapas más complejos o realizar análisis espaciales.

A continuación, se te pedirá que decidas si deseas o no descargar los datos. Si eliges hacerlo, luego deberás seleccionar el formato deseado. Sigue las instrucciones en pantalla.

In [ ]:
opcion_descarga = input("¿Quieres descargar los datos? (si/no): ")
if opcion_descarga.lower() == "si":
    formatos = ["jpeg", "csv", "shp"]
    formato_seleccionado = input(f"¿En qué formato deseas descargar? Opciones: {', '.join(formatos)}: ")

    if formato_seleccionado == "jpeg":
        descargar_jpeg(nombre_especie_input, fig)
    elif formato_seleccionado == "csv":
        descargar_csv(df)
    elif formato_seleccionado == "shp":
        descargar_shp(df)
    else:
        print("Formato no reconocido.")